## First we import the packages that are necessary for our visualization

In [1]:
%matplotlib inline
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets
import cartopy
import numpy as np

## Then we read the data from the csv files
There are several cells below are data managing and cleaning work for array_vals and locations.
Zeyu borrowed these codes from Prof. Turk's code example

In [2]:
array_vals = pd.read_csv("data-readonly/transportable_array/data_tohoku_norm_transpose.csv", header=None)

In [3]:
# codes from instructor week 10 example
# set the time index to the array_vals
v = pd.date_range("2:46PM", "6:46PM", freq="1s")
v -= v[0]
array_vals["time"] = v
array_vals.set_index("time", inplace=True)

In [4]:
#codes from instructor's week 10 example
#read the location data, delete the column a and b which are useless
locations = pd.read_csv("/srv/nbgrader/data/transportable_array/location.txt",
                        delimiter="\t", names =["longitude", "latitude", "a", "b"])


del locations["a"], locations["b"]

## Now we use the cartopy function to get the state from the US.

In [5]:

states = cartopy.io.shapereader.natural_earth(resolution='110m', category='cultural',
                                    name='admin_1_states_provinces_lakes_shp')

In [6]:
#code from instrucotr's week 11 example
reader =cartopy.io.shapereader.Reader(states)
all_states = list(reader.records())

In [7]:
geometries = [state.geometry for state in all_states]

## Now we compute the distance between the detector and the tohoku quake
Xiaoye used haversine function to compute the distance between two coordinates.
* The main reason we use function provided by Prof. Turk is we thought this is easiest way to compute.
* The addvantage of this approach is that the way is intuitive
* The disadvantage of this approach is that the high time and space complexity since we allocate a list and used a for loop to compute and append the value

In [8]:
#conpute the distance between detectors and the tohoku
from haversine import haversine
distance=[]
for i in range(438):
    distance.append(haversine((locations["latitude"][i],locations["longitude"][i]),(142.369,38.322)))
distance

[4166.932746904038,
 4224.519467418433,
 4252.128890076834,
 2284.9671603305246,
 3117.8895703102235,
 3895.4675180574104,
 3970.250870905521,
 4029.5274726369244,
 4109.878006932558,
 4172.788428197611,
 4228.6206559339225,
 4288.685275904362,
 4361.722643418009,
 4421.551471872628,
 4507.446017062583,
 4548.356531381905,
 2698.997163741373,
 3920.2649070211564,
 3987.8455394303987,
 4128.945927735803,
 4192.253729323637,
 4252.496543497155,
 4308.70871096099,
 4369.793456426028,
 4444.253748427942,
 4509.628098650106,
 4575.698204622288,
 3941.4128555865245,
 4006.721624440434,
 4080.12182993061,
 4126.913980611313,
 4214.134338124893,
 4262.773195471178,
 4330.02694358507,
 4384.827709015359,
 4451.681022805205,
 4523.679856079039,
 4589.562402215125,
 3956.3453894985964,
 4026.086714546656,
 4087.602802576863,
 4157.55623160486,
 4212.746738343071,
 4274.627746473273,
 4334.8933286976935,
 4407.847635345877,
 4475.1465888870025,
 4543.386177880373,
 4598.043643472942,
 4690.8597960

## Now we generate a new dataframe that is filled with the waveform sorted by distance

In [9]:
#assign the distance value to the locations table with column name "distance"
locations["distance"]=distance

In [10]:
# a new table called s_by_distance. it is the table contains the same values with sorted by the distance
# assign the table called newarray. it is the table that transpose the array_vals and sorted by the distance
# with help by Prof. Turk
s_by_distance = locations.sort_values("distance")
newarray=array_vals.T.iloc[s_by_distance.index.values]

## Then we generate a 2D array, with 438 rows and each list has 14400 logged values 
* The reason We use this approach is that the the looged value will perform better for following spectrogram
* The weakness of this approach is it will take long time to complete and the Jupyter will give us runtime warning so the time complexity would be high
* The strengh: We do this step in here so that we can avoid this step in following plotting method that will make the interact plot move extremely slow

In [12]:
newplotArray=np.log10(newarray**2)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log10
  """Entry point for launching an IPython kernel.


## Now we are ready to plot this interactive plot
Zeyu and Xiaoye made the plot individually and then connecte the plots to a whole interactive plot. We used matplotlib and the ipywidgets to do that.
* Reason we used these approaches: 1.matplotlib is easier for scattering the detector dots on the US Map. Bqplot may do better, but it had some scale problem when we tried to plot the dot on the map and it was trickier for fixing that. ipywidets is convenient for interact it is easy to understand for code reviewer.
* Strengths: 1.The plot is staightforward;it is easy to understand and it shows the all the usefull information. 2.The interactive works well and it shows the changes of the plot when you change the value of the time and the stations. 3. Our spectrogram is more intuitive and more aethetic than spectrograms showing the oringinal value. 4. The layout of the plot is similair as the project requierments. 5.We made the inslide station rather than drop down. So, you can easily change the value of the station without scrolling down a long list to find a specific value.
* weakness: 1.long runtime. 2.The x axis of line plot and the x axis of spectrogram is not same. 3. Some overload dots on the map
* expectations: 1. We tried to make the plot smaller but it was hard to show the whole plot in the screen no matter how we change the axes. 2. We tried to lable the cross line of the spectrogram so that the lable will move following the movement of crossline, but we didn't find a right way to do that. 3. We thought if we change the color of the plot but at last we didn't think it is necessary. 4.We tried to make the x axis consitent but the jupyter gave as error when we change the xlims,set the the datetime etc. Maybe we didn't cover something. 

In [13]:
#code of the interactive plot
@ipywidgets.interact(time=(0,14400,1),station=(0,437,1))
def plotting(time,station):
    #map, defining its size, location
    fig = plt.figure(figsize=(6.5,6.5))
    ax = fig.add_axes([0.0, 0.0, 1.0, 1.0],projection = cartopy.crs.LambertConformal())
    ax.set_extent([-125, -66.5, 20, 50], cartopy.crs.Geodetic())
    ax.add_geometries(geometries, crs = cartopy.crs.PlateCarree(), zorder=0)
    #plotting the scatter
    ax.scatter(locations ["longitude"], locations["latitude"], transform = cartopy.crs.Geodetic(),c=array_vals.iloc[time])
    ax.set_title("Map",fontsize=30)
    ax = plt.axes([1.2, 0.6, 0.5, 0.4])
    #plotting the spectrogram using the 2D array, set the aspect to auto so the plot won't be small
    ax.imshow(newplotArray,aspect='auto')
    ax.set_ylim(0,438)
    #plot the crossline with help of Prof. Turk
    ax.plot( [time, time], [0,438], '-k')    
    ax.plot( [0, 14400], [station, station], 'k')
    ax.set_title("Spectrogram of Detector",fontsize=18)
    ax.set_xlabel("time",fontsize=15)
    ax.set_ylabel("detector",fontsize=15)
    ax = plt.axes([1.2, 0.05, 0.5, 0.4])
    #Plot the line plot
    ax.plot(array_vals[station])
    ax.set_title("Line Plot of waveform",fontsize=18)
    ax.set_xlabel("time",fontsize=15)
    ax.set_ylabel("wave value",fontsize=15)
#     #plt.xlim(0,14400)
    plt.ylim(-1.0, 1.0)

interactive(children=(IntSlider(value=7200, description='time', max=14400), IntSlider(value=218, description='station', max=437), Output()), _dom_classes=('widget-interact',))